# NLP para calificaciones DE MOMENTO ESTE NO SIRVE

In [2]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

AttributeError: module 'numpy.linalg._umath_linalg' has no attribute '_ilp64'

**1. Preparacion de datos**

In [ ]:
# 1. Preparación de Datos
data = pd.read_csv('data/normalized_merged_data.csv')
data['text'] = data['text'].str.lower().str.replace('[^a-z\s]', '')

train, test = train_test_split(data, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)

tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

train_encodings = tokenizer(train['text'].tolist(), truncation=True, padding='max_length', max_length=25, return_tensors='tf')
val_encodings = tokenizer(val['text'].tolist(), truncation=True, padding='max_length', max_length=25, return_tensors='tf')
test_encodings = tokenizer(test['text'].tolist(), truncation=True, padding='max_length', max_length=25, return_tensors='tf')

**2. Definicion del modelo**

In [ ]:
# 2. Definición del Modelo
base_model = TFGPT2LMHeadModel.from_pretrained("gpt2-medium")
input_layer = tf.keras.layers.Input(shape=(25,), dtype=tf.int32)
sequence_output = base_model(input_layer)[0]
cls_token = sequence_output[:, 0, :]
wording_head = tf.keras.layers.Dense(1, activation='sigmoid', name='wording')(cls_token)
model = tf.keras.models.Model(inputs=input_layer, outputs=wording_head)
model.compile(optimizer='adam', loss='mean_squared_error')


**3. Entrenamiento**

In [ ]:
# 3. Entrenamiento
model.fit(train_encodings['input_ids'], train['normalized_wording'], validation_data=(val_encodings['input_ids'], val['normalized_wording']), epochs=5)

In [ ]:
# 3.5 Guardar modelo
model.save('nlp_gpt2_wording.h5')

**4. Evaluacion**

In [ ]:
# 4. Evaluación
losses = model.evaluate(test_encodings['input_ids'], test['normalized_wording'])

**5. Prediccion**

In [ ]:
# 5. Predicción
new_text = ["The Third Wave was an experiment to see how people reacted to a new one leader government."]
new_encodings = tokenizer(new_text, truncation=True, padding='max_length', max_length=25, return_tensors='tf')
predicted_wording = model.predict(new_encodings['input_ids'])